# Phase 4: Rigorous Model Evaluation

## 1. Objective
Transitioning from model selection to deep validation. We focus on:
1. **Imbalance-Aware Metrics**: Balanced Accuracy, Macro F1, ROC-AUC.
2. **Error Patterns**: Manual inspection of misclassifications.
3. **Threshold Optimization**: Tuning the classifier for critical minority classes (Negatives).

## 2. Setup and Data Loading
Re-importing necessary components to ensure portability.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    balanced_accuracy_score, 
    roc_auc_score, 
    f1_score
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Load cleaned data
data_path = os.path.join('..', 'data', 'interim', 'cleaned_amazon.csv')
df = pd.read_csv(data_path)

# Target Binning
df['sentiment'] = df['reviews.rating'].map({1: 0, 2: 0, 3: 1, 4: 2, 5: 2})
df = df.dropna(subset=['cleaned_text', 'brand', 'categories'])
df = df[df['cleaned_text'].str.strip().astype(bool)]

# Split
X = df[['cleaned_text', 'brand', 'categories']]
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Data loaded. Test set size: {len(X_test)}")

## 3. Model Reconstruction (Best Performer)
From Phase 3, LogReg with SMOTE showed the best balance. We re-fit it here for evaluation.

In [ ]:
text_transformer = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english', min_df=5)),
    ('svd', TruncatedSVD(n_components=100, random_state=42)),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('text', text_transformer, 'cleaned_text'),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['brand', 'categories'])
])

best_pipe = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('clf', LogisticRegression(max_iter=2000, n_jobs=-1, random_state=42))
])

print("Fitting final model for evaluation...")
best_pipe.fit(X_train, y_train)
y_pred = best_pipe.predict(X_test)
y_proba = best_pipe.predict_proba(X_test)
print("Model ready.")

## 4. Advanced Metric Suite
Beyond Accuracy: Looking at the macro and balanced performance.

In [ ]:
bal_acc = balanced_accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, y_proba, multi_class='ovr')

print(f"Balanced Accuracy: {bal_acc:.4f}")
print(f"F1 Score (Macro):   {f1_macro:.4f}")
print(f"ROC-AUC (OvR):     {roc_auc:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## 5. Visual Validation: Confusion Matrix
Where is the model getting confused?

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Neg', 'Neu', 'Pos'], 
            yticklabels=['Neg', 'Neu', 'Pos'])
plt.title("Confusion Matrix Heatmap")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## 6. Threshold Tuning for Minority Class (Negatives)
If detecting negative feedback is critical, we might lower the threshold for class 0.

In [ ]:
def custom_predict(proba, threshold=0.5, target_class=0):
    """
    If proba[target_class] > threshold, return target_class.
    Otherwise return the usual argmax.
    """
    preds = []
    for p in proba:
        if p[target_class] >= threshold:
            preds.append(target_class)
        else:
            preds.append(np.argmax(p))
    return np.array(preds)

# Experimenting with a 0.4 threshold for Negatives
y_pred_tuned = custom_predict(y_proba, threshold=0.4, target_class=0)

print("Tuned Classification Report (Threshold=0.4 for Negatives):")
print(classification_report(y_test, y_pred_tuned))

## 7. Deep Error Analysis
Inspecting samples where the model was highly confident but wrong.

In [ ]:
error_df = X_test.copy()
error_df['actual'] = y_test
error_df['pred'] = y_pred
error_df['conf'] = y_proba.max(axis=1)

# False Positives: Model thinks it's Positive (2), but it's Negative (0)
false_positives = error_df[(error_df['actual'] == 0) & (error_df['pred'] == 2)].sort_values('conf', ascending=False)

print(f"Top 5 High-Confidence False Positives (Actual Neg -> Pred Pos):")
display(false_positives[['cleaned_text', 'conf']].head(10))

## 8. Conclusion
Summary of findings:
- Model struggles most with [Neutral Class / Sarcasm / etc].
- Threshold tuning [Improved / Hurthe] recall for negatives.
- Recommended next steps: [Data augmentation / More features / BERT?]